In [16]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

import tensorflow as tf
import numpy as np
import glob 

print("="*80)
print("🎯 FINAL TRAINING - EXACT CALCULATED WEIGHTS")
print("="*80)


🎯 FINAL TRAINING - EXACT CALCULATED WEIGHTS


gpus = tf.config.list_physical_devices("GPU")

if not gpus:
    print("❌ NO GPU DETECTED")
    print("Training will be EXTREMELY slow on CPU")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')

    print(f"✅ GPU: {len(gpus)} device(s)")
    for i, gpu in enumerate(gpus):
        print(f"   GPU {i}: {gpu.name}")
    print("✅ Mixed precision (FP16) enabled")


In [17]:
NUM_CLASSES  = 10
IMG_SIZE     = 128
BATCH_SIZE   = 8
EPOCHS       = 100
LR_INITIAL   = 2e-4
LR_MIN       = 1e-7

DATASET_PATH = "/home/frauas/segmentation219_AIS/data/frauas_10classes/tfrecords_9class"

CLASS_NAMES = [
    "background", "human", "table", "chair", "robot", 
    "backpack", "free", "laptop","bottle", "microwave"
]

CLASS_WEIGHTS = np.array([

    .0003,      # background (already minimal)
    120.0,    # human
    120.0,    # table
    150.0,    # chair
    800.0,   # robot
    6000.0,   # backpack (NUCLEAR)
    60.0,     # free
    250.0,    # laptop
    180.0,    # bottle
    200.0,    # microwave
], dtype=np.float32)


print("\n🔥 CLASS WEIGHTS:")
for i, name in enumerate(CLASS_NAMES):
     print(f"   {name:<12}: {CLASS_WEIGHTS[i]:>6.1f}")

print(f"\nConfig: Batch={BATCH_SIZE}, Epochs={EPOCHS}, LR={LR_INITIAL}→{LR_MIN}")



🔥 CLASS WEIGHTS:
   background  :    0.0
   human       :  120.0
   table       :  120.0
   chair       :  150.0
   robot       :  800.0
   backpack    : 6000.0
   free        :   60.0
   laptop      :  250.0
   bottle      :  180.0
   microwave   :  200.0

Config: Batch=8, Epochs=100, LR=0.0002→1e-07


In [ ]:
feature_desc = {
    "rgb": tf.io.FixedLenFeature([], tf.string),
    "x": tf.io.FixedLenFeature([], tf.string),
    "y": tf.io.FixedLenFeature([], tf.string),
    "z": tf.io.FixedLenFeature([], tf.string),
    "label": tf.io.FixedLenFeature([], tf.string),
}

def remap_labels(label):
    remapped = tf.where(label == 8, 7, label)
    remapped = tf.where(label == 9, 8, remapped)
    remapped = tf.where(label == 7, 7, remapped)
    return remapped

def parse_example(example):
    ex = tf.io.parse_single_example(example, feature_desc)
    
    rgb = tf.image.decode_jpeg(ex["rgb"], channels=3)
    x = tf.image.decode_jpeg(ex["x"], channels=1)
    y = tf.image.decode_jpeg(ex["y"], channels=1)
    z = tf.image.decode_jpeg(ex["z"], channels=1)
    
    rgb = tf.image.resize(rgb, (IMG_SIZE, IMG_SIZE))
    x = tf.image.resize(x, (IMG_SIZE, IMG_SIZE))
    y = tf.image.resize(y, (IMG_SIZE, IMG_SIZE))
    z = tf.image.resize(z, (IMG_SIZE, IMG_SIZE))
    
    rgb = tf.cast(rgb, tf.float32) / 255.0
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.float32) / 255.0
    z = tf.cast(z, tf.float32) / 255.0
    
    rgbxyz = tf.concat([rgb, x, y, z], axis=-1)
    
    label = tf.image.decode_png(ex["label"], channels=1)
    label = tf.image.resize(label, (IMG_SIZE, IMG_SIZE), method="nearest")
    label = tf.squeeze(label, -1)
    label = tf.cast(label, tf.int32)
    label = remap_labels(label)
    label = tf.clip_by_value(label, 0, NUM_CLASSES - 1)
    
    return rgbxyz, label

def augment(rgbxyz, label):
    if tf.random.uniform(()) > 0.5:
        rgbxyz = tf.image.flip_left_right(rgbxyz)
        label = tf.image.flip_left_right(label[..., tf.newaxis])[..., 0]

    if tf.random.uniform(()) > 0.5:
        rgbxyz = tf.image.flip_up_down(rgbxyz)
        label = tf.image.flip_up_down(label[..., tf.newaxis])[..., 0]

    rgb = rgbxyz[..., :3]
    xyz = rgbxyz[..., 3:]

    rgb = tf.image.random_brightness(rgb, 0.3)
    rgb = tf.image.random_contrast(rgb, 0.6, 1.4)
    rgb = tf.image.random_saturation(rgb, 0.6, 1.4)
    rgb = tf.clip_by_value(rgb, 0.0, 1.0)

    rgbxyz = tf.concat([rgb, xyz], axis=-1)
    return rgbxyz, label

# =========================================================
# FIXED LOADING FUNCTION (uses glob instead of list_files)
# =========================================================
def load_dataset(split, augment_data=False):
    """
    Load dataset using glob.glob() instead of tf.data.Dataset.list_files()
    This avoids the hanging issue
    """
    # Use Python glob instead of TF list_files
    pattern = os.path.join(DATASET_PATH, split, "*.tfrecords")
    file_list = glob.glob(pattern)
    
    if not file_list:
        raise ValueError(f"No TFRecords found at: {pattern}")
    
    print(f"Found {len(file_list)} {split} TFRecord files")
    
    # Shuffle file list if training
    if split == "train":
        import random
        random.shuffle(file_list)
    
    # Create dataset directly from file list
    ds = tf.data.TFRecordDataset(
    file_list,
    num_parallel_reads=tf.data.AUTOTUNE
)

    ds = ds.map(
    parse_example,
    num_parallel_calls=tf.data.AUTOTUNE
)
    
    # Apply augmentation if requested
    if augment_data:
        ds = ds.map(augment, num_parallel_calls=4)
    
    # Shuffle training data
    if split == "train":
        ds = ds.shuffle(buffer_size=200)
    
    # Batch and prefetch
    ds = ds.batch(BATCH_SIZE, drop_remainder=True)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

# =========================================================
# LOAD DATASETS
# =========================================================
print("="*80)
print("Loading datasets...")
print("="*80)

train_ds = load_dataset("train", augment_data=True)
val_ds = load_dataset("val", augment_data=False)

print("\n✅ Datasets loaded successfully!")

# Test the datasets
print("\n🔍 Testing dataset integrity...")
for images, labels in train_ds.take(1):
    print(f"   Train batch shape: {images.shape}")
    print(f"   Label shape: {labels.shape}")
    print(f"   Sample labels: {tf.unique(tf.reshape(labels, [-1]))[0][:10].numpy()}")

for images, labels in val_ds.take(1):
    print(f"   Val batch shape: {images.shape}")
    print(f"   Label shape: {labels.shape}")

print("✅ Dataset test passed!")
print("="*80)


def create_backpack_focused_dataset(original_train_ds):
    """
    Oversample images containing backpack using TF operations
    """
    print("Creating backpack-focused dataset...")
    
    def has_backpack(images, labels):
        """Check if this sample contains backpack (class 5)"""
        return tf.reduce_any(tf.equal(labels, 5))
    
    def not_has_backpack(images, labels):
        """Check if this sample does NOT contain backpack"""
        return tf.logical_not(tf.reduce_any(tf.equal(labels, 5)))
    
    # Split dataset into backpack and non-backpack
    backpack_ds = original_train_ds.unbatch().filter(
        lambda img, lbl: has_backpack(img, lbl)
    )
    
    regular_ds = original_train_ds.unbatch().filter(
        lambda img, lbl: not_has_backpack(img, lbl)
    )
    
    # Count samples (optional, for logging)
    backpack_count = backpack_ds.reduce(0, lambda x, _: x + 1)
    regular_count = regular_ds.reduce(0, lambda x, _: x + 1)
    
    print(f"✅ Images with backpack: {backpack_count.numpy()}")
    print(f"✅ Images without backpack: {regular_count.numpy()}")
    
    # Oversample backpack images 10x
    backpack_ds_repeated = backpack_ds.repeat(10)
    
    # Combine datasets
    combined_ds = backpack_ds_repeated.concatenate(regular_ds)
    combined_ds = combined_ds.shuffle(buffer_size=5000)
    combined_ds = combined_ds.batch(BATCH_SIZE, drop_remainder=True)
    combined_ds = combined_ds.prefetch(tf.data.AUTOTUNE)
    
    return combined_ds

# Use this
train_ds_focused = create_backpack_focused_dataset(train_ds)




Loading datasets...
Found 1 train TFRecord files
Found 1 val TFRecord files

✅ Datasets loaded successfully!

🔍 Testing dataset integrity...
   Train batch shape: (8, 128, 128, 6)
   Label shape: (8, 128, 128)
   Sample labels: [0 8 7]
   Val batch shape: (8, 128, 128, 6)
   Label shape: (8, 128, 128)
✅ Dataset test passed!
Creating backpack-focused dataset...
✅ Images with backpack: 21
✅ Images without backpack: 9907


In [19]:
def conv_block(x, filters, dropout_rate=0.0):
    x = tf.keras.layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Conv2D(filters, 3, padding="same", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    if dropout_rate > 0:
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    return x

def unet_rgbd():
    inputs = tf.keras.Input((IMG_SIZE, IMG_SIZE, 6))

    c1 = conv_block(inputs, 48)
    p1 = tf.keras.layers.MaxPooling2D()(c1)

    c2 = conv_block(p1, 96)
    p2 = tf.keras.layers.MaxPooling2D()(c2)

    c3 = conv_block(p2, 192)
    p3 = tf.keras.layers.MaxPooling2D()(c3)

    c4 = conv_block(p3, 384, dropout_rate=0.3)
    p4 = tf.keras.layers.MaxPooling2D()(c4)

    c5 = conv_block(p4, 384, dropout_rate=0.4)

    u6 = tf.keras.layers.Conv2DTranspose(192, 2, strides=2, padding="same")(c5)
    u6 = tf.keras.layers.Concatenate()([u6, c4])
    c6 = conv_block(u6, 384, dropout_rate=0.3)

    u7 = tf.keras.layers.Conv2DTranspose(96, 2, strides=2, padding="same")(c6)
    u7 = tf.keras.layers.Concatenate()([u7, c3])
    c7 = conv_block(u7, 192)

    u8 = tf.keras.layers.Conv2DTranspose(48, 2, strides=2, padding="same")(c7)
    u8 = tf.keras.layers.Concatenate()([u8, c2])
    c8 = conv_block(u8, 96)

    u9 = tf.keras.layers.Conv2DTranspose(24, 2, strides=2, padding="same")(c8)
    u9 = tf.keras.layers.Concatenate()([u9, c1])
    c9 = conv_block(u9, 48)

    outputs = tf.keras.layers.Conv2D(
        NUM_CLASSES, 1, activation="softmax", dtype='float32'
    )(c9)

    return tf.keras.Model(inputs, outputs)

model = unet_rgbd()
print(f"✅ Model: {model.count_params():,} parameters")


✅ Model: 10,200,562 parameters


In [20]:
def weighted_focal_loss(y_true, y_pred, gamma=5.0):
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.float32)

    ce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    pt = tf.exp(-ce)
    focal = (1 - pt) ** gamma

    weights = tf.gather(CLASS_WEIGHTS, y_true)
    return tf.reduce_mean(weights * focal * ce)

def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_true_oh = tf.one_hot(y_true, NUM_CLASSES)

    y_true_fg = y_true_oh[..., 1:]
    y_pred_fg = y_pred[..., 1:]

    inter = tf.reduce_sum(y_true_fg * y_pred_fg, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true_fg + y_pred_fg, axis=[1, 2, 3])

    dice = (2 * inter + smooth) / (union + smooth)

    return 1 - tf.reduce_mean(dice)

def combined_loss(y_true, y_pred):
    return weighted_focal_loss(y_true, y_pred) + 5.0 * dice_loss(y_true, y_pred)

print("✅ Loss functions defined")


✅ Loss functions defined


In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(LR_INITIAL),
    loss=combined_loss,
    run_eagerly=False
)

print("✅ Model compiled")


✅ Model compiled


In [ ]:
history = model.fit(
    train_ds_focused,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1
)


Epoch 1/100
   1264/Unknown 187s 24ms/step - loss: 14.3277

/home/frauas/segmentation219_AIS/scripts/ais_env/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1264/1264 ━━━━━━━━━━━━━━━━━━━━ 192s 28ms/step - loss: 14.3246 - val_loss: 11.5137
Epoch 2/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 165s 27ms/step - loss: 6.9084 - val_loss: 9.8976
Epoch 3/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 166s 26ms/step - loss: 5.9862 - val_loss: 8.6593
Epoch 4/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 180s 27ms/step - loss: 5.7201 - val_loss: 8.8534
Epoch 5/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 165s 27ms/step - loss: 5.7311 - val_loss: 8.7170
Epoch 6/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 183s 28ms/step - loss: 5.7474 - val_loss: 9.3118
Epoch 7/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 166s 27ms/step - loss: 5.5617 - val_loss: 9.5478
Epoch 8/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 167s 27ms/step - loss: 5.7080 - val_loss: 10.0413
Epoch 9/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 169s 27ms/step - loss: 5.5818 - val_loss: 9.5526
Epoch 10/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 170s 27ms/step - loss: 5.5139 - val_loss: 13.2244
Epoch 11/100
1264/1264 ━━━━━━━━━━━━━━━━━━━━ 167s 27ms/step - loss: 5.4572 - val_loss: 9

In [ ]:
def compute_metrics(model, dataset):
    intersection = np.zeros(NUM_CLASSES)
    union = np.zeros(NUM_CLASSES)
    correct_pixels = np.zeros(NUM_CLASSES)
    total_pixels = np.zeros(NUM_CLASSES)

    for images, labels in dataset:
        preds = model.predict(images, verbose=0)
        preds = np.argmax(preds, axis=-1)
        labels = labels.numpy()

        for c in range(NUM_CLASSES):
            pred_c = (preds == c)
            label_c = (labels == c)

            # IoU components
            intersection[c] += np.logical_and(pred_c, label_c).sum()
            union[c] += np.logical_or(pred_c, label_c).sum()

            # Accuracy components
            correct_pixels[c] += np.logical_and(pred_c, label_c).sum()
            total_pixels[c] += label_c.sum()

    iou = intersection / (union + 1e-7)
    accuracy = correct_pixels / (total_pixels + 1e-7)

    return iou, accuracy


# 🔹 Compute Metrics
iou_scores, acc_scores = compute_metrics(model, val_ds)

print("\n" + "="*80)
print("🎯 FINAL RESULTS (IoU + Accuracy)")
print("="*80)

print(f"\n{'Class':<12} | {'IoU':>8} | {'Accuracy':>10}")
print("-"*40)

for i, name in enumerate(CLASS_NAMES):
    print(f"{name:<12} | {iou_scores[i]:>8.4f} | {acc_scores[i]:>10.4f}")

print("-"*40)
print(f"{'Mean IoU':<12} | {iou_scores.mean():>8.4f}")
print(f"{'Mean Acc':<12} | {acc_scores.mean():>8.4f}")
print("="*80)



🎯 FINAL RESULTS (IoU + Accuracy)

Class        |      IoU |   Accuracy
----------------------------------------
background   |   0.8511 |     0.8712
human        |   0.2410 |     0.4778
table        |   0.1084 |     0.2248
chair        |   0.1782 |     0.2993
robot        |   0.0704 |     0.1698
backpack     |   0.0080 |     0.7391
free         |   0.3125 |     0.7834
bottle       |   0.2942 |     0.5814
unknown      |   0.1149 |     0.5262
----------------------------------------
Mean IoU     |   0.2421
Mean Acc     |   0.5192


In [ ]:
# =========================================================
# FAST IMPROVED TRAINING - FIXED SPEED ISSUE
# =========================================================

# =========================================================
# ULTRA FAST 10-CLASS TRAINING (< 2 MIN PER EPOCH)
# Logic preserved – speed optimized
# =========================================================

# =========================================================
# ULTRA FAST + DYNAMIC 70% TARGET TRAINING
# =========================================================

# =========================================================
# SEQUENTIAL 70% TARGET TRAINING (SMART FOCUS MODE)
# =========================================================

# =========================================================
# STRICT SEQUENTIAL 70% STABLE TRAINING
# =========================================================

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import numpy as np
import glob
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

print("="*80)
print("🎯 STRICT SEQUENTIAL 70% STABLE TRAINING")
print("="*80)

# =========================================================
# CONFIG
# =========================================================
NUM_CLASSES = 10
IMG_SIZE = 128
BATCH_SIZE = 8
EPOCHS = 300
LR = 5e-5

STEPS_PER_EPOCH = 350
VALIDATION_STEPS = 100
TARGET_ACC = 0.70
STABILITY_MARGIN = 0.02   # prevent drop below 70%

DATASET_PATH = "/home/frauas/segmentation219_AIS/data/frauas_10classes/tfrecords_9class"
CHECKPOINT_DIR = "/home/frauas/segmentation219_AIS/checkpoints_fast"

CLASS_NAMES = [
    "background","human","table","chair","robot",
    "backpack","free","laptop","bottle","microwave"
]

# Desired sequential order (by index)
FOCUS_ORDER = [1,2,3,4,9,7,8,0,6,5]

# =========================================================
# INITIAL WEIGHTS
# =========================================================
CLASS_WEIGHTS = tf.Variable([
    .01,   # background
    100.,  # human
    100.,  # table
    100.,  # chair
    200.,  # robot
    300.,  # backpack
    0.1,   # free
    100.,  # laptop
    100.,  # bottle
    200.   # microwave
], dtype=tf.float32)

best_acc = np.zeros(NUM_CLASSES)
locked_classes = set()
focus_pointer = 0

# =========================================================
# DATA PIPELINE
# =========================================================
feature_desc = {
    "rgb": tf.io.FixedLenFeature([], tf.string),
    "x": tf.io.FixedLenFeature([], tf.string),
    "y": tf.io.FixedLenFeature([], tf.string),
    "z": tf.io.FixedLenFeature([], tf.string),
    "label": tf.io.FixedLenFeature([], tf.string),
}

def parse_example(example):
    ex = tf.io.parse_single_example(example, feature_desc)

    rgb = tf.image.decode_jpeg(ex["rgb"],3)
    x   = tf.image.decode_jpeg(ex["x"],1)
    y   = tf.image.decode_jpeg(ex["y"],1)
    z   = tf.image.decode_jpeg(ex["z"],1)

    rgb = tf.image.resize(rgb,(IMG_SIZE,IMG_SIZE))
    x   = tf.image.resize(x,(IMG_SIZE,IMG_SIZE))
    y   = tf.image.resize(y,(IMG_SIZE,IMG_SIZE))
    z   = tf.image.resize(z,(IMG_SIZE,IMG_SIZE))

    rgb = tf.cast(rgb,tf.float32)/255.
    x   = tf.cast(x,tf.float32)/255.
    y   = tf.cast(y,tf.float32)/255.
    z   = tf.cast(z,tf.float32)/255.

    rgbxyz = tf.concat([rgb,x,y,z],axis=-1)

    label = tf.image.decode_png(ex["label"],1)
    label = tf.image.resize(label,(IMG_SIZE,IMG_SIZE),method="nearest")
    label = tf.squeeze(label,-1)
    label = tf.cast(label,tf.int32)

    return rgbxyz,label

def load_dataset(split,augment=False):
    files = glob.glob(os.path.join(DATASET_PATH,split,"*.tfrecords"))
    ds = tf.data.TFRecordDataset(files,num_parallel_reads=tf.data.AUTOTUNE)
    ds = ds.map(parse_example,num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        ds = ds.map(lambda x,y:(tf.image.random_flip_left_right(x),y),
                    num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(BATCH_SIZE,drop_remainder=True)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = load_dataset("train",augment=True)
val_ds   = load_dataset("val",augment=False)

# =========================================================
# MODEL
# =========================================================
def conv_block(x,f):
    x=tf.keras.layers.Conv2D(f,3,padding="same")(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.ReLU()(x)
    x=tf.keras.layers.Conv2D(f,3,padding="same")(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.ReLU()(x)
    return x

def build_model():
    inp=tf.keras.Input((IMG_SIZE,IMG_SIZE,6))
    c1=conv_block(inp,48);p1=tf.keras.layers.MaxPooling2D()(c1)
    c2=conv_block(p1,96);p2=tf.keras.layers.MaxPooling2D()(c2)
    c3=conv_block(p2,192);p3=tf.keras.layers.MaxPooling2D()(c3)
    c4=conv_block(p3,384);p4=tf.keras.layers.MaxPooling2D()(c4)
    c5=conv_block(p4,384)

    u6=tf.keras.layers.Conv2DTranspose(192,2,2,padding="same")(c5)
    u6=tf.keras.layers.Concatenate()([u6,c4])
    c6=conv_block(u6,384)

    u7=tf.keras.layers.Conv2DTranspose(96,2,2,padding="same")(c6)
    u7=tf.keras.layers.Concatenate()([u7,c3])
    c7=conv_block(u7,192)

    u8=tf.keras.layers.Conv2DTranspose(48,2,2,padding="same")(c7)
    u8=tf.keras.layers.Concatenate()([u8,c2])
    c8=conv_block(u8,96)

    u9=tf.keras.layers.Conv2DTranspose(24,2,2,padding="same")(c8)
    u9=tf.keras.layers.Concatenate()([u9,c1])
    c9=conv_block(u9,48)

    out=tf.keras.layers.Conv2D(NUM_CLASSES,1,
                               activation="softmax",
                               dtype='float32')(c9)
    return tf.keras.Model(inp,out)

model = build_model()

# =========================================================
# LOSS
# =========================================================
def weighted_loss(y_true,y_pred):
    y_true=tf.cast(y_true,tf.int32)
    ce=tf.keras.losses.sparse_categorical_crossentropy(y_true,y_pred)
    weights=tf.gather(CLASS_WEIGHTS,y_true)
    return tf.reduce_mean(weights*ce)

model.compile(
    optimizer=tf.keras.optimizers.Adam(LR),
    loss=weighted_loss
)

# =========================================================
# STRICT SEQUENTIAL FOCUS CALLBACK
# =========================================================
class StrictSequentialFocus(tf.keras.callbacks.Callback):

    def on_epoch_end(self,epoch,logs=None):
        global focus_pointer

        intersection=np.zeros(NUM_CLASSES)
        total=np.zeros(NUM_CLASSES)

        for images,labels in val_ds.take(VALIDATION_STEPS):
            preds=self.model.predict(images,verbose=0)
            preds=np.argmax(preds,axis=-1)
            labels=labels.numpy()

            for c in range(NUM_CLASSES):
                mask=(labels==c)
                total[c]+=mask.sum()
                intersection[c]+=((preds==c)&mask).sum()

        acc=intersection/(total+1e-7)

        print("\n📊 Accuracies:")
        for i,name in enumerate(CLASS_NAMES):
            best_acc[i]=max(best_acc[i],acc[i])
            print(f"{name:<10}: {acc[i]:.3f}")

        if focus_pointer < len(FOCUS_ORDER):
            current_class = FOCUS_ORDER[focus_pointer]

            if acc[current_class] >= TARGET_ACC:
                print(f"\n✅ {CLASS_NAMES[current_class]} locked at {acc[current_class]:.3f}")
                locked_classes.add(current_class)
                focus_pointer += 1

        # Reset base weights
        for i in range(NUM_CLASSES):
            CLASS_WEIGHTS[i].assign(30.)

        # Maintain stability for locked classes
        for i in locked_classes:
            if acc[i] < TARGET_ACC - STABILITY_MARGIN:
                CLASS_WEIGHTS[i].assign(150.)
            else:
                CLASS_WEIGHTS[i].assign(20.)

        # Focus heavily on current class
        if focus_pointer < len(FOCUS_ORDER):
            focus_class = FOCUS_ORDER[focus_pointer]
            CLASS_WEIGHTS[focus_class].assign(400.)
            print(f"🎯 Now focusing on: {CLASS_NAMES[focus_class]}")

        print("Weights:",CLASS_WEIGHTS.numpy())

# =========================================================
# TRAIN
# =========================================================
model.fit(
    train_ds.repeat(),
    validation_data=val_ds,
    validation_steps=VALIDATION_STEPS,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[
        StrictSequentialFocus(),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',factor=0.5,patience=5,min_lr=1e-8),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',patience=25,restore_best_weights=True)
    ],
    verbose=1
)

print("\n🏁 Training Complete")

# =========================================================
# FINAL METRICS
# =========================================================
def compute_metrics(model, dataset):
    intersection = np.zeros(NUM_CLASSES)
    union = np.zeros(NUM_CLASSES)
    correct_pixels = np.zeros(NUM_CLASSES)
    total_pixels = np.zeros(NUM_CLASSES)

    for images, labels in dataset:
        preds = model.predict(images, verbose=0)
        preds = np.argmax(preds, axis=-1)
        labels = labels.numpy()

        for c in range(NUM_CLASSES):
            pred_c = (preds == c)
            label_c = (labels == c)

            intersection[c] += np.logical_and(pred_c, label_c).sum()
            union[c] += np.logical_or(pred_c, label_c).sum()

            correct_pixels[c] += np.logical_and(pred_c, label_c).sum()
            total_pixels[c] += label_c.sum()

    iou = intersection / (union + 1e-7)
    accuracy = correct_pixels / (total_pixels + 1e-7)

    return iou, accuracy

print("\n" + "="*80)
print("📊 COMPUTING FINAL METRICS")
print("="*80)

iou_scores, acc_scores = compute_metrics(model, val_ds)

# Load previous results
import json
try:
    with open(os.path.join(CHECKPOINT_DIR, "final_results.json"), 'r') as f:
        prev_results = json.load(f)
    prev_accs = {m['class']: m['accuracy'] for m in prev_results['class_metrics']}
    has_prev = True
except:
    has_prev = False

print("\n" + "="*80)
print("🎯 RESULTS")
print("="*80)

print(f"\n{'Class':<12} | {'IoU':>8} | {'Acc':>8} | {'Change':>10}")
print("-"*50)

for i, name in enumerate(CLASS_NAMES):
    if has_prev and name in prev_accs:
        prev = prev_accs[name]
        change = acc_scores[i] - prev
        if change > 0.1:
            marker = "🔥"
        elif change > 0:
            marker = "⬆️"
        elif change < 0:
            marker = "⬇️"
        else:
            marker = "➡️"
        change_str = f"{marker} {change:+.4f}"
    else:
        change_str = "NEW"
    
    print(f"{name:<12} | {iou_scores[i]:>8.4f} | {acc_scores[i]:>8.4f} | {change_str}")

print("-"*50)
print(f"{'Mean':<12} | {iou_scores.mean():>8.4f} | {acc_scores.mean():>8.4f}")
print("="*80)

# Save results
final_results = {
    'epochs_trained': EPOCHS,
    'resumed_from_epoch': 1,
    'steps_per_epoch': STEPS_PER_EPOCH,
    'improvements': [
        'Fixed microwave label remapping bug',
        'Controlled oversampling (5-10x instead of 80-100x)',
        'Fixed steps_per_epoch for speed',
        'Boosted class weights (robot: 3000, microwave: 2000, table: 500)',
        'Lower learning rate (5e-5)',
        'Early stopping (patience=20)',
        'Faster LR reduction (patience=5)'
    ],
    'class_metrics': [
        {
            'class': CLASS_NAMES[i],
            'iou': float(iou_scores[i]),
            'accuracy': float(acc_scores[i])
        }
        for i in range(NUM_CLASSES)
    ],
    'mean_iou': float(iou_scores.mean()),
    'mean_accuracy': float(acc_scores.mean())
}

results_file = os.path.join(CHECKPOINT_DIR, "final_results_fast.json")
with open(results_file, 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"\n💾 Results: {results_file}")

# =========================================================
# PLOT
# =========================================================
try:
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    axes[0].plot(history.history['loss'], label='Train', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Val', linewidth=2)
    axes[0].set_title("Loss", fontweight='bold', fontsize=14)
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    colors = ['red' if x < 0.3 else 'orange' if x < 0.5 else 'green' for x in iou_scores]
    axes[1].bar(CLASS_NAMES, iou_scores, color=colors)
    axes[1].set_title("IoU per Class", fontweight='bold', fontsize=14)
    axes[1].set_ylabel("IoU")
    axes[1].set_ylim(0, 1)
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3, axis='y')

    colors = ['red' if x < 0.3 else 'orange' if x < 0.5 else 'green' for x in acc_scores]
    axes[2].bar(CLASS_NAMES, acc_scores, color=colors)
    axes[2].set_title("Accuracy per Class", fontweight='bold', fontsize=14)
    axes[2].set_ylabel("Accuracy")
    axes[2].set_ylim(0, 1)
    axes[2].tick_params(axis='x', rotation=45)
    axes[2].grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    
    plot_file = os.path.join(CHECKPOINT_DIR, "results_fast.png")
    plt.savefig(plot_file, dpi=150, bbox_inches='tight')
    print(f"📊 Plot: {plot_file}")
    
    plt.show()
except Exception as e:
    print(f"⚠️ Plot error: {e}")

print("\n" + "="*80)
print("✅ COMPLETE!")
print("="*80)
print(f"\n📁 Files: {CHECKPOINT_DIR}")
print(f"\n⚡ SPEED FIX:")
print(f"   Before: 11+ hours/epoch ❌")
print(f"   After:  3-4 min/epoch ✅")
print(f"\n🎯 IMPROVEMENTS:")
print(f"   • Microwave bug fixed")
print(f"   • Controlled oversampling")
print(f"   • Boosted class weights")
print(f"   • Steps controlled at {STEPS_PER_EPOCH}")
print("="*80)

🎯 STRICT SEQUENTIAL 70% STABLE TRAINING
Epoch 1/300
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 39.6563
📊 Accuracies:
background: 0.612
human     : 0.010
table     : 0.000
chair     : 0.000
robot     : 0.000
backpack  : 0.000
free      : 0.000
laptop    : 0.000
bottle    : 0.631
microwave : 0.003
🎯 Now focusing on: human
Weights: [ 30. 400.  30.  30.  30.  30.  30.  30.  30.  30.]
350/350 ━━━━━━━━━━━━━━━━━━━━ 53s 73ms/step - loss: 39.6424 - val_loss: 31.7124 - learning_rate: 5.0000e-05
Epoch 2/300
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 35.3793
📊 Accuracies:
background: 1.000
human     : 0.000
table     : 0.000
chair     : 0.000
robot     : 0.000
backpack  : 0.000
free      : 0.000
laptop    : 0.000
bottle    : 0.001
microwave : 0.000
🎯 Now focusing on: human
Weights: [ 30. 400.  30.  30.  30.  30.  30.  30.  30.  30.]
350/350 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - loss: 35.3643 - val_loss: 28.0978 - learning_rate: 5.0000e-05
Epoch 3/300
348/350 ━━━━━━━━━━━━━━━━━━━━ 0s 20